In [2]:
import boto3
import json
import datetime
from IPython.display import display, JSON

bedrock_agent = boto3.client("bedrock-agent")
bedrock_runtime = boto3.client("bedrock-runtime")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

model_id = "amazon.nova-micro-v1:0"

In [3]:
prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        {
            "name": "v1",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7,
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])


==================== Response Object ====================



<IPython.core.display.JSON object>


==================== Prompt ARN ====================

arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ


In [5]:
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier="arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ"
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ:1


In [6]:
prompt_arn_with_version = "arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])


==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**
We are seeking a passionate and innovative UX Designer to join our dynamic team. As a UX Designer, you will play a pivotal role in crafting intuitive and engaging user interfaces. This is a full-time position available either in our New York office or remotely. 

**Responsibilities:**
- Design compelling user interfaces that enhance user experience.
- Conduct and analyze usability testing to inform design decisions.
- Collaborate closely with cross-functional product teams to ensure alignment and deliver high-quality products.

**Requirements:**
- Minimum of 3+ years of experience in UX design.
- Proficiency in Figma, HTML, and CSS.
- Excellent communication skills to articulate design concepts and gather feedback.

**Location:** New York, NY or Remote

Join our diverse and inclusive team to shape the future of user experience design. We celebrate differences and strive for an equitable w

In [7]:
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== OPTIMIZED PROMPT ====================

"# HR Assistant: Job Description Generator\n\n## Task\nCreate a professional, inclusive job description based on the provided information.\n\n## Inputs\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Guidelines\n- Begin with a compelling, concise summary of the role (1-2 sentences)\n- Use gender-neutral, inclusive language throughout\n- Organize content with clear headings (About the Role, Key Responsibi

In [9]:
prompt_identifier = "arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ"

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = "# HR Assistant: Job Description Generator\n\n## Task\nCreate a professional, inclusive job description based on the provided information.\n\n## Inputs\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n## Guidelines\n- Begin with a compelling, concise summary of the role (1-2 sentences)\n- Use gender-neutral, inclusive language throughout\n- Organize content with clear headings (About the Role, Key Responsibilities, Qualifications, etc.)\n- Focus on essential requirements to encourage diverse applicants\n- Highlight company values and commitment to inclusion\n- Include location and work arrangement details clearly\n- Use active voice and action verbs\n- Avoid industry jargon or acronyms\n- Maintain a professional yet approachable tone\n\n## Format Requirements\n- Total length: Maximum 250 words\n- Structure: Summary paragraph followed by bulleted lists for responsibilities and requirements\n- End with a brief call to action for candidates\n\nGenerate the job description immediately without any preamble or additional explanation."


response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== UPDATED PROMPT ====================

# HR Assistant: Job Description Generator

## Task
Create a professional, inclusive job description based on the provided information.

## Inputs
### Job Title
{{job_title}}

### Responsibilities
{{responsibilities}}

### Requirements
{{requirements}}

### Location
{{location}}

### Work Type
{{work_type}}

## Guidelines
- Begin with a compelling, concise summary of the role (1-2 sentences)
- Use gender-neutral, inclusive language throughout
- Organize content with clear headings (About the Role, Key Responsibilities, Qualifications, etc

In [10]:
response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ:2


In [11]:
prompt_arn = "arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Summary:**
We are seeking a passionate and creative UX Designer to join our dynamic team. You will play a pivotal role in designing user interfaces, conducting usability testing, and collaborating closely with our product teams to enhance user experience. This is a full-time position that offers flexibility with options to work from New York or remotely.

**Responsibilities:**
- Develop and design intuitive, user-friendly interfaces.
- Conduct comprehensive usability testing to identify and resolve user experience issues.
- Collaborate with cross-functional teams to ensure seamless product development.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma for design prototyping.
- Knowledge of HTML/CSS for implementing design solutions.
- Strong communication skills to effectively convey ideas and collaborate with team members.

**Location:** New York or remote

**Work T

In [12]:
prompt_arn = "arn:aws:bedrock:us-east-1:142728997126:prompt/4G63WB4AZQ:2"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**UX Designer**

**About the Role**
Join our team as a UX Designer to craft intuitive user interfaces that enhance user experience. This role involves running usability tests and collaborating with cross-functional teams to deliver top-notch digital products.

**Key Responsibilities**
- Design compelling user interfaces for web and mobile applications
- Conduct usability testing to refine and improve user experiences
- Work closely with product, design, and engineering teams to bring innovative ideas to life

**Qualifications**
- 3+ years of experience in UX design
- Proficiency in Figma
- Knowledge of HTML/CSS
- Strong communication skills

**Location and Work Type**
This full-time position is based in New York or can be fulfilled remotely. We value diverse perspectives and are committed to fostering an inclusive workplace.

**Call to Action**
If you're passionate about creating seamless user experiences and ready to grow wit

In [13]:
response = bedrock_agent.list_prompts()

for prompt in response['promptSummaries']:
    prompt_id = prompt['id']  # The correct key is 'id', not 'promptId'
    print(f"Deleting prompt: {prompt['name']} (ID: {prompt_id})")
    bedrock_agent.delete_prompt(promptIdentifier=prompt_id)

print("All prompts deleted successfully")

Deleting prompt: job-description-20251230233827 (ID: 4G63WB4AZQ)
All prompts deleted successfully
